In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# **Moment invariant**

Reading image file

In [11]:
img1 = cv2.imread("Segmented_Teeth/segmented tooth1_1.png")
img2 = cv2.imread("Segmented_Teeth/segmented tooth1_2.png")
img3 = cv2.imread("Segmented_Teeth/segmented_tooth1_3.png")

In [12]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

In [13]:
moments1 = cv2.HuMoments(cv2.moments(img1)).flatten()
print("ORIGINAL MOMENTS for img 1: {}".format(moments1))

moments2 = cv2.HuMoments(cv2.moments(img2)).flatten()
print("ORIGINAL MOMENTS for img 2: {}".format(moments2))

moments3 = cv2.HuMoments(cv2.moments(img3)).flatten()
print("ORIGINAL MOMENTS for img 3: {}".format(moments3))

ORIGINAL MOMENTS for img 1: [9.31673872e-04 3.94367501e-07 1.16589393e-10 1.50416801e-11
 6.29560476e-22 9.44115823e-15 2.07860839e-23]
ORIGINAL MOMENTS for img 2: [9.32487012e-04 4.19574583e-07 7.85269033e-11 1.14095140e-11
 3.37913841e-22 7.05015323e-15 4.94653700e-23]
ORIGINAL MOMENTS for img 3: [1.18052474e-03 9.45728897e-07 1.96083982e-10 8.24694512e-11
 1.04041240e-20 7.77455385e-14 1.31763320e-21]


In [14]:
# for i in range(0,7):
#    moments1[i] = -np.sign(1.0, moments1[i])*np.math.log10(np.abs(moments1[i]))
   
#-1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))

lmoment1 = np.abs(np.log10(moments1))
print('log transformation for moments1',lmoment1)
lmoment2 = np.abs(np.log10(moments2))
print('log transformation for moments2', lmoment2)
lmoment3 = np.abs(np.log10(moments3))
print('log transformation for moments3', lmoment3)

log transformation for moments1 [ 3.03073608  6.40409888  9.93334096 10.82270365 21.20096254 14.02497472
 22.68222732]
log transformation for moments2 [ 3.03035721  6.37719083 10.10498153 10.94273285 21.47119402 14.15180144
 22.30569874]
log transformation for moments3 [ 2.92792491  6.02423334  9.70755788 10.0837069  19.98279448 13.10932452
 20.88020547]


In [15]:
dist = np.linalg.norm(lmoment1 - lmoment2)
print('Distance between moment1 and moment2',dist)

Distance between moment1 and moment2 0.5248576217835638


In [17]:
dist = np.linalg.norm(lmoment1 - lmoment3)
print('Distance between moment1 and moment3',dist)

Distance between moment1 and moment3 2.5142779577407346


# *CNN classification*

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(2,activation = "softmax")   #Adding the Output Layer
])

In [12]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [13]:
bs=30         #Setting batch size
train_dir = "training dataset"   #Setting training directory
validation_dir = "validation dataset"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

Found 26 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [16]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    # steps_per_epoch=150 // bs,
                    epochs=10,
                    # validation_steps=50 // bs,
                    verbose=2)

Epoch 1/10
1/1 - 1s - loss: 0.1311 - acc: 0.9231 - val_loss: 0.3037 - val_acc: 0.8571 - 1s/epoch - 1s/step
Epoch 2/10
1/1 - 2s - loss: 0.1024 - acc: 0.9615 - val_loss: 0.1910 - val_acc: 0.9286 - 2s/epoch - 2s/step
Epoch 3/10
1/1 - 2s - loss: 0.0592 - acc: 0.9615 - val_loss: 0.3783 - val_acc: 0.8571 - 2s/epoch - 2s/step
Epoch 4/10
1/1 - 1s - loss: 0.0311 - acc: 1.0000 - val_loss: 0.5836 - val_acc: 0.8571 - 1s/epoch - 1s/step
Epoch 5/10
1/1 - 1s - loss: 0.0169 - acc: 1.0000 - val_loss: 0.6565 - val_acc: 0.8571 - 1s/epoch - 1s/step
Epoch 6/10
1/1 - 1s - loss: 0.0201 - acc: 1.0000 - val_loss: 0.6771 - val_acc: 0.8571 - 1s/epoch - 1s/step
Epoch 7/10
1/1 - 2s - loss: 0.0037 - acc: 1.0000 - val_loss: 0.6469 - val_acc: 0.8571 - 2s/epoch - 2s/step
Epoch 8/10
1/1 - 2s - loss: 0.0091 - acc: 1.0000 - val_loss: 0.5065 - val_acc: 0.8571 - 2s/epoch - 2s/step
Epoch 9/10
1/1 - 1s - loss: 2.5738e-04 - acc: 1.0000 - val_loss: 0.3961 - val_acc: 0.8571 - 1s/epoch - 1s/step
Epoch 10/10
1/1 - 1s - loss: 7.48